# (G12) 번역가는 대화에도 능하다
****
# Project: 멋진 챗봇 만들기

## Step 1. 데이터 로드
```c
$ git clone https://github.com/songys/Chatbot_data.git
$ mv Chatbot_data ~/aiffel/transformer_chatbot 
```

In [163]:
# 필요한 라이브러리 선언
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


In [164]:
df = pd.read_csv("/home/aiffel-dj20/aiffel/transformer_chatbot/Chatbot_data/ChatbotData .csv")
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [187]:
questions = df.iloc[:, ['Q']]
anwsers = df.iloc[: ,['A']]

IndexError: .iloc requires numeric indexers, got ['Q']

In [184]:
questions

,Q
0,12시 땡!
1,1지망 학교 떨어졌어
2,3박4일 놀러가고 싶다
4,PPL 심하네
5,SD카드 망가졌어
...,...
11815,회식 중이라고 하는데 연락이 안돼.
11817,후회 없이 사랑하고 싶어
11818,훔쳐보는 것도 눈치 보임.
11820,흑기사 해주는 짝남.


## Step 2. 데이터 정제

- ```preprocess_sentence()``` 함수
> 1. 영문자는 모두 소문자 변환.
> 2. 영문자와 한글, 숫자 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거.

In [185]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 a-z A-Z?.!,0-9]+", " ", sentence)
    
    return sentence
print("done")

done


In [186]:
preprocess_sentence(questions)

AttributeError: 'DataFrame' object has no attribute 'lower'

## Step 3. 데이터 토큰화

- 토큰화에는 KoNLPy의 ```mecab```클래스를 사용.
- 아래 조건을 만족하는 ```build_corpus()```함수를 구현.
> 1. 소스 문장 데이터와 타겟 문장 데이터를 입력으로 받음.
> 2. 데이터를 앞서 정의한 ```preprocess_sentence()```함수로 정제하고, 토큰화.
> 3. 토큰화는 전달받은 토크나이즈 함수를 사용. 이번엔 mecab.morphs 함수를 전달.
> 4. 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외.
> 5. 중복되는 문장은 데이터에서 제외합니다. ```소스 : 타겟``` 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사. 중복 쌍이 흐트러지지 않도록 유의.


- 구현한 함수를 활용하여 ```questions```와 ```answers```를 각각 ```que_corpus```, ```ans_corpus```에 토큰화하여 저장.

In [161]:
from konlpy.tag import Mecab

que_corpus = []
ans_corpus = []

def build_corpus(src, tgt):
    tokenizer = Mecab()
    max_len = 30
    
    src = preprocess_sentence(src)
    src = tokenizer.morphs(src)
    tgt = preprocess_sentence(tgt)
    tgt = tokenizer.morphs(tgt)
    
    #que_corpus.append(s for s in src if src < max_len)
    #ans_corpus.append(t for t in tgt if tgt < max_len)
    
    que_corpus.append(src)
    ans_corpus.append(tgt)
    
    return que_corpus, ans_corpus
    #for sen1, sen2 in zip(src, tgt):
        #if len(src) <= max_len:
            #que_corpus.append(sen1)
            #ans_corpus.append(sen2)
    #return que_corpus, ans_corpus

print("done")

done


In [162]:
que_corpus, ans_corpus = build_corpus(questions, answers)

print(que_corpus[:5])

[['0', '12', '시', '땡', '!', '1', '1', '지망', '학교', '떨어졌', '어', '2', '3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다', '3', '3', '박', '4', '일', '정도', '놀', '러', '가', '고', '싶', '다', '4', 'ppl', '심하', '네', '.', '.', '.', '11818', '훔쳐', '보', '는', '것', '도', '눈치', '보임', '.', '11819', '훔쳐', '보', '는', '것', '도', '눈치', '보임', '.', '11820', '흑기사', '해', '주', '는', '짝', '남', '.', '11821', '힘든', '연애', '좋', '은', '연애', '라는', '게', '무슨', '차이', '일까', '?', '11822', '힘들', '어서', '결혼', '할까봐', 'name', 'q', ',', 'length', '11823', ',', 'dtype', 'object']]


In [155]:
print(ans_corpus[:5])

[['0', '하루', '가', '또', '가', '네요', '.', '1', '위', '로', '해', '드립니다', '.', '2', '여행', '은', '언제나', '좋', '죠', '.', '3', '여행', '은', '언제나', '좋', '죠', '.', '4', '눈살', '이', '찌푸려', '지', '죠', '.', '.', '.', '.', '11818', '티', '가', '나', '니까', '눈치', '가', '보이', '는', '거', '죠', '!', '11819', '훔쳐', '보', '는', '거', '티나', '나', '봐요', '.', '11820', '설렜', '겠', '어요', '.', '11821', '잘', '헤어질', '수', '있', '는', '사이', '여부', '인', '거', '같', '아요', '.', '11822', '도피성', '결혼', '은', '하', '지', '않', '길', '바라', '요', '.', 'name', 'a', ',', 'length', '11823', ',', 'dtype', 'object']]


## Step 4. Augmentation

- Lexical Substitution을 실제로 적용해.
```c
https://drive.google.com/u/0/ucid=0B0ZXk88koS2KbDhXdWg1Q2RydlU&export=download 
```


- 다운로드한 모델을 활용해 데이터를 Augmentation. 앞서 정의한 ```lexical_sub()```함수를 참고.
- Augmentation된 ```que_corpus```와 원본```ans_corpus```가 병렬을 이루도록, 이후엔 반대로 원본```que_corpus```와 Augmentation된```ans_corpus```가 병렬을 이루도록 하여 전체 데이터가 원래의 3배가량으로 늘어나도록 함.

In [43]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [154]:
# Lexical Substitution 구현하기
def lexical_sub(sentence, word2vec):
    import random

    res = ""
    toks = sentence.split()

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [ ]:
from tqdm import tqdm_notebook

new_corpus = []


for idx in tqdm_notebook(range(3000)):
    old_src = tokenizer.decode_ids(src_corpus[idx])

    new_src = lexical_sub(old_src, wv)

    if new_src is not None: new_corpus.append(new_src)

    new_corpus.append(old_src)

print(new_corpus[:10])